In [41]:
import requests

# URL base da API
BASE_URL = "https://metadata.idl.ucsf.edu/solr/ltdl3/query"

def get_total_documents():
    """
    Obtém a quantidade total de documentos disponíveis na API Solr.

    Returns:
        int: Número total de documentos disponíveis.
    """
    params = {
        "q": "*:*",  # Busca geral para contar todos os documentos
        "rows": 0,   # Não retorna documentos, apenas a contagem
        "wt": "json" # Resposta em formato JSON
    }

    # Requisição à API
    response = requests.get(BASE_URL, params=params)
    if response.status_code == 200:
        # Extrai o número total de documentos
        total_documents = response.json().get("response", {}).get("numFound", 0)
        return total_documents
    else:
        # Trata erros na requisição
        print(f"Erro {response.status_code} ao consultar o total de documentos.")
        print(f"Mensagem de erro: {response.text}")
        return None

# Exemplo de uso
total_documents = get_total_documents()
if total_documents is not None:
    print(f"Quantidade total de documentos disponíveis: {total_documents}")
else:
    print("Falha ao obter o total de documentos.")

Quantidade total de documentos disponíveis: 21359679


In [42]:
import math

def calculate_iterations(total_documents, max_results):
    """
    Calcula o número de iterações necessárias para processar todos os documentos.

    Args:
        total_documents (int): O número total de documentos disponíveis na API.
        max_results (int): O número máximo de documentos por lote (tamanho do lote).

    Returns:
        int: O número total de iterações necessárias.
    """
    if max_results <= 0:
        raise ValueError("max_results deve ser maior que zero.")
    if total_documents <= 0:
        raise ValueError("total_documents deve ser maior que zero.")
    
    # Calcula o número total de iterações
    iterations = math.ceil(total_documents / max_results)
    return iterations

# Exemplo de uso
try:
    # total_documents = total_documents  # Número total de documentos (exemplo da API)
    max_results = 100  # Tamanho do lote (exemplo)

    iterations = calculate_iterations(total_documents, max_results)
    print(f"Para processar {total_documents} documentos com lotes de {max_results}, serão necessárias {iterations} iterações.")
except ValueError as e:
    print(f"Erro: {e}")

Para processar 21359679 documentos com lotes de 100, serão necessárias 213597 iterações.


In [43]:
import requests
import sqlite3
import time
from datetime import datetime

# URL base da API
BASE_URL = "https://metadata.idl.ucsf.edu/solr/ltdl3/query"
DB_PATH = "document_comparisons.db"

def initialize_database():
    """
    Cria ou atualiza a tabela Collections_Types no banco de dados.
    """
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    # Criação ou atualização da tabela
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Collections_Types (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        collection TEXT NOT NULL,
        type TEXT NOT NULL,
        count INTEGER NOT NULL,
        UNIQUE(collection, type)
    )
    """)
    conn.commit()
    conn.close()

In [44]:
def save_collection_type(collection, doc_type, count):
    """
    Salva ou atualiza uma combinação collection/type com sua contagem de documentos.
    """
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    try:
        cursor.execute("""
        INSERT INTO Collections_Types (collection, type, count)
        VALUES (?, ?, ?)
        ON CONFLICT(collection, type) DO UPDATE SET count=count + ?;
        """, (collection, doc_type, count, count))
    except sqlite3.IntegrityError:
        pass  # Evita duplicatas

    conn.commit()
    conn.close()

In [45]:
def fetch_combinations():
    """
    Busca as combinações de collections e types diretamente em lotes usando cursorMark
    e registra a contagem de documentos.
    """
    start_time = datetime.now()
    cursor_mark = "*"  # Cursor inicial
    max_results = 100
    iteration = 0

    while True:
        params = {
            "q": "*:*",               # Busca geral
            "fl": "collection,type",  # Retorna apenas os campos desejados
            "wt": "json",             # Formato da resposta
            "rows": max_results,      # Tamanho do lote
            "cursorMark": cursor_mark,  # Paginação com cursor
            "sort": "id asc"          # Ordenação necessária para usar cursorMark
        }

        response = requests.get(BASE_URL, params=params)

        if response.status_code == 200:
            data = response.json()
            docs = data.get("response", {}).get("docs", [])
            next_cursor_mark = data.get("nextCursorMark", cursor_mark)

            if not docs:
                break  # Sai do loop se não houver mais documentos

            # Processa as combinações de collection e type
            counts = {}
            for doc in docs:
                collections = doc.get("collection")
                doc_types = doc.get("type")

                # Trata collection e type como listas
                if not isinstance(collections, list):
                    collections = [collections]
                if not isinstance(doc_types, list):
                    doc_types = [doc_types]

                # Conta as combinações
                for collection in collections:
                    for doc_type in doc_types:
                        if collection and doc_type:
                            key = (collection, doc_type)
                            counts[key] = counts.get(key, 0) + 1

            # Salva cada combinação no banco de dados com a contagem
            for (collection, doc_type), count in counts.items():
                save_collection_type(collection, doc_type, count)

            # Atualiza progresso
            iteration += 1
            elapsed_time = datetime.now() - start_time
            percent_complete = (iteration * max_results / 21359679) * 100  # Substitua 21359679 pelo total de documentos se disponível
            print(f"\rProcessando... {percent_complete:.2f}% concluído. Tempo decorrido: {elapsed_time}", end="")

            # Atualiza o cursor para a próxima iteração
            if cursor_mark == next_cursor_mark:  # Se não houver progresso no cursor, encerra
                break
            cursor_mark = next_cursor_mark

        elif response.status_code == 403:
            print(f"Erro 403: {response.text}")
            print("Aguardando 2 segundos antes de continuar...")
            time.sleep(2)

        else:
            print(f"Erro {response.status_code} ao buscar combinações.")
            print(f"Mensagem de erro: {response.text}")
            break

    print("\nProcessamento concluído!")

In [ ]:
# Inicializar banco de dados e executar busca
initialize_database()
fetch_combinations()

Processando... 3.59% concluído. Tempo decorrido: 4:50:14.256171

In [52]:
import pandas as pd

conn = sqlite3.connect(DB_PATH)
# Verificar se a tabela existe e carregar os dados
query_check = "SELECT name FROM sqlite_master WHERE type='table' AND name='Collections_Types';"
if not pd.read_sql_query(query_check, conn).empty:
    # Lê os dados da tabela no DataFrame do Pandas
    df = pd.read_sql_query("SELECT * FROM Collections_Types;", conn)

    # Verifica se há dados na tabela
    if not df.empty:
        print(f"Tabela 'Collections_Types' contém {len(df)} registros:")
    else:
        print("A tabela 'Collections_Types' está vazia.")
else:
    print("A tabela 'Collections_Types' não existe no banco de dados.")

# Fechar a conexão
conn.close()
df

Tabela 'Collections_Types' contém 218 registros:


,id,collection,type,count
0,1,Council for Tobacco Research Records,agenda,1
1,2,Master Settlement Agreement,agenda,3
2,3,RJ Reynolds Records,letter,13
3,4,RJ Reynolds Records,list,1
4,5,Master Settlement Agreement,letter,29
...,...,...,...,...
213,267,British American Tobacco Records,chart,1
214,268,British American Tobacco Records,report,2
215,269,British American Tobacco Records,bibliography,1
216,270,British American Tobacco Records,table,1


In [36]:
def drop_table():
    """
    Remove a tabela Collections_Types do banco de dados.
    """
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    try:
        # Dropar a tabela sem verificação prévia
        cursor.execute("DROP TABLE IF EXISTS Collections_Types;")
        conn.commit()
        print("Tabela 'Collections_Types' foi removida (se existia).")
    except Exception as e:
        print(f"Erro ao dropar a tabela: {e}")
    finally:
        conn.close()

In [53]:
# Exemplo de uso
drop_table()

Tabela 'Collections_Types' foi removida (se existia).
